# Dataset

In [ ]:
# import glob


# def frame_number(image_path):
#     """ Extracts frame number from filepath """
#     image_path = image_path.replace('\\','/')
#     try:
#         return int(image_path.split('/')[-1].split('.jpg')[0])
#     except:
#         print("Got error while getting image number ....")
#         exit()

# all_img = sorted(glob.glob('../input/alzheimer-dataset/nano_blue_all/nano_blue_all/100109/*.jpg'), key=lambda path: frame_number(path))


In [ ]:
# all_img

In [1]:
!pip install gdown

In [2]:
!gdown 'https://drive.google.com/uc?id=1j_aPp2V_XksVT4zjdLX2knAPoG2Lgrmn' -O 'best.pth'

In [3]:
# import os
# import glob
# import math
# import time
# import random
# import pandas as pd
# import numpy as np

# import torch
# import torchvision
# import torch.nn as nn
# from torch.utils.data.dataset import Dataset
# from torchvision.transforms import transforms
# from torch.optim.lr_scheduler import OneCycleLR
# import cv2
# #from torch.utils.tensorboard import SummaryWriter

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import matthews_corrcoef, accuracy_score, log_loss


# # #%% [code]
# class VideoIterator(Dataset):
#     def __init__(self, df, transforms, device, aug=False):
#         self.df = df
#         print(self.df["stalled"].value_counts())
#         self.transforms = transforms
#         self.device = device
#         self.aug = aug
#         self.rotations = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
#         self.flips = [0, 1, -1]
        
#     def _frame_number(self, image_path):
#         """ Extracts frame number from filepath """
#         image_path = image_path.replace('\\','/')
#         try:
#             return int(image_path.split('/')[-1].split('.jpg')[0])
#         except:
#             print("Got error while getting image number ....")
#             exit()
        
#     def __getitem__(self, index):
#         row = self.df.iloc[index]
#         x = []
#         x_back=[]
#         #print(row["path"])
#         dir_path = row["path"].split('.mp4')[0]


#         all_img_path = sorted(glob.glob(f'{dir_path}/*.jpg'), key=lambda path: self._frame_number(path))
#         all_img_path_back = sorted(glob.glob(f'{dir_path}/*.jpg'), key=lambda path: self._frame_number(path), reverse=True)

        
# #         video = cv2.VideoCapture(row["path"])
# #         if not video.isOpened():
# #             print("Error opening video file.")
#         for for_img, back_img in zip(all_img_path, all_img_path_back):
#             frame1 = cv2.imread(for_img)
#             #frame1 = cv2.resize(frame1, (112,112), interpolation=cv2.INTER_CUBIC)
#             frame2 = cv2.imread(back_img)
#             #frame2 = cv2.resize(frame2, (112,112), interpolation=cv2.INTER_CUBIC)
            
#             x.append(frame1)
#             x_back.append(frame2)
#         #video.release()
        
#         if self.aug:    
#             if random.random() <= 0.25:
#                 rot = random.choice(self.rotations)
#                 x = [cv2.rotate(img, rot) for img in x]
#                 x_back = [cv2.rotate(img, rot) for img in x_back]
#             if random.random() <= 0.25:
#                 fl = random.choice(self.flips)
#                 x = [cv2.flip(img, fl) for img in x]
#                 x_back = [cv2.flip(img, fl) for img in x_back]
        
#         x = [self.transforms(frame) for frame in x]
#         x_back = [self.transforms(frame) for frame in x_back]
        
#         # print(type(x))
#         #print(x)
#         # print(f'shape of x {x.shape}')
#         x = torch.stack(x)
#         x_back = torch.stack(x_back)
#         #print("\n\n\n first size ", x.size(), "\n\n\n\n")
#         x = x.permute(1, 0, 2, 3) #Buji nai 
#         x_back = x_back.permute(1, 0, 2, 3) #Buji nai 
#         #print("\n\n\n second size ", x.size(), "\n\n\n\n")
#         x = x.unsqueeze(0)
#         x_back = x_back.unsqueeze(0)
#         #print("\n\n\n third size ", x.size(), "\n\n\n\n")
#         x = x.to(self.device, dtype=torch.float)
#         #print("\n\n\n\n X size ", x.size(), "\n\n")
#         x_back = x_back.to(self.device, dtype=torch.float)
#         #print("\n\n\n\n x_back size ", x_back.size(), "\n\n")
#         y = torch.FloatTensor([[row["stalled"]]]).to(self.device)
#         return x,x_back, y

#     def __len__(self):
#         return len(self.df)

#     def shuffle(self):
#         self.df = self.df.sample(frac=1).reset_index(drop=True)



# Dataset (Point Cloud)

In [10]:
import os
import glob
import math
import time
import random
import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
from torch.utils.data.dataset import Dataset
from torchvision.transforms import transforms
from torch.optim.lr_scheduler import OneCycleLR
import cv2
#from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, accuracy_score, log_loss


# #%% [code]
class VideoIterator(Dataset):
    def __init__(self, df, transforms, device, aug=False):
        self.df = df
        print(self.df["stalled"].value_counts())
        self.transforms = transforms
        self.device = device
        self.aug = aug
        self.rotations = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
        self.flips = [0, 1, -1]
        

        
    def __getitem__(self, index):
        row = self.df.iloc[index]
        x = []
        x_back=[]
        #print(row["path"])
        dir_path = f"{row['path'].split('.mp4')[0]}.pt"


        x = torch.load(dir_path)
        x_back = torch.flip(x, [1])
        
        x = x.unsqueeze(0)
        x_back = x_back.unsqueeze(0)
        #print("\n\n\n third size ", x.size(), "\n\n\n\n")
        x = x.to(self.device, dtype=torch.float)
        #print("\n\n\n\n X size ", x.size(), "\n\n")
        x_back = x_back.to(self.device, dtype=torch.float)
        #print("\n\n\n\n x_back size ", x_back.size(), "\n\n")
        y = torch.FloatTensor([[row["stalled"]]]).to(self.device)
        return x, x_back, y

    def __len__(self):
        return len(self.df)

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)



# Model

In [1]:
class R2plus1dModel(nn.Module):
    def __init__(self):
        super(R2plus1dModel, self).__init__()

        cnn = torchvision.models.video.r2plus1d_18(pretrained=True)
        self.cnn_pc = nn.Sequential(*list(cnn.children())[:-1])
        self.fc = nn.Linear(in_features=1024,
                                out_features=1)
        self.sig = nn.Sigmoid()

    def forward(self, input1, input2):
        x = self.cnn_pc(input1)
        x_back = self.cnn_pc(input2)
        x = torch.cat((x, x_back), 1).squeeze().unsqueeze(0)
        x = self.fc(x)
        x = self.sig(x)
        return x

# Dataset + Model + Training

In [12]:

# #%% [code]
# import os
# import cv2
# import math
# import time
# import random
# import pandas as pd
# import numpy as np

# import torch
# import torchvision
# import torch.nn as nn
# from torch.utils.data.dataset import Dataset
# from torchvision.transforms import transforms
# from torch.optim.lr_scheduler import OneCycleLR
# #from torch.utils.tensorboard import SummaryWriter

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import matthews_corrcoef, accuracy_score, log_loss


# #%% [code]
# class VideoIterator(Dataset):
#     def __init__(self, df, transforms, device, aug=False):
#         self.df = df
#         print(self.df["stalled"].value_counts())
#         self.transforms = transforms
#         self.device = device
#         self.aug = aug
#         self.rotations = [cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180]
#         self.flips = [0, 1, -1]
        
#     def __getitem__(self, index):
#         row = self.df.iloc[index]
#         x = []
#         #print(row["path"])
#         video = cv2.VideoCapture(row["path"])
#         if not video.isOpened():
#             print("Error opening video file.")
#         while video.isOpened():
#             ret, frame = video.read()
#             if ret:
#                 x.append(frame)
#                 #print(f' return {ret}')
#             else:
#                 #print(f' return {ret}')
#                 break
#         video.release()
        
#         if self.aug:    
#             if random.random() <= 0.25:
#                 rot = random.choice(self.rotations)
#                 x = [cv2.rotate(img, rot) for img in x]
#             if random.random() <= 0.25:
#                 fl = random.choice(self.flips)
#                 x = [cv2.flip(img, fl) for img in x]
        
#         x = [self.transforms(frame) for frame in x]
#         # print(type(x))
#         #print(x)
#         # print(f'shape of x {x.shape}')
#         x = torch.stack(x)
#         #print("\n\n\n first size ", x.size(), "\n\n\n\n")
#         x = x.permute(1, 0, 2, 3) #Buji nai 
#         #print("\n\n\n second size ", x.size(), "\n\n\n\n")
#         x = x.unsqueeze(0)
#         #print("\n\n\n third size ", x.size(), "\n\n\n\n")
#         x = x.to(self.device, dtype=torch.float)
#         y = torch.FloatTensor([[row["stalled"]]]).to(self.device)
#         return x, y

#     def __len__(self):
#         return len(self.df)

#     def shuffle(self):
#         self.df = self.df.sample(frac=1).reset_index(drop=True)


# #%% [code]
# # class R2plus1dModel(nn.Module):
# #     def __init__(self):
# #         super(R2plus1dModel, self).__init__()

# #         self.cnn = torchvision.models.video.r2plus1d_18(pretrained=True)
# #         self.cnn.fc = nn.Linear(in_features=512,
# #                                 out_features=1)
# #         self.sig = nn.Sigmoid()

# #     def forward(self, input):
# #         x = self.cnn(input)
# #         x = self.sig(x)
# #         return x
    

# class Rmc3Model(nn.Module):
#     def __init__(self):
#         super(Rmc3Model, self).__init__()

#         self.cnn = torchvision.models.video.mc3_18(pretrained=True)
#         self.cnn.fc = nn.Linear(in_features=512,
#                                 out_features=1)
#         self.sig = nn.Sigmoid()

#     def forward(self, input):
#         x = self.cnn(input)
#         x = self.sig(x)
#         return x


# class R3dModel(nn.Module):
#     def __init__(self):
#         super(R3dModel, self).__init__()

#         self.cnn = torchvision.models.video.r3d_18(pretrained=True)
#         self.cnn.fc = nn.Linear(in_features=512,
#                                 out_features=1)
#         self.sig = nn.Sigmoid()

#     def forward(self, input):
#         x = self.cnn(input)
#         x = self.sig(x)
#         return x


# #%% [code]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


#%% [code]
data_folder = "../input/alzheimer-point-cloud-unfixed/nano_voxel_otsu_unfixed"
train_data_folder = data_folder + "/"
train_csv = data_folder + "train_data.csv"


#%% [code]
transformations = transforms.Compose([
                  transforms.ToTensor(),
                  transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])
])
    
# df = pd.read_csv(train_csv)
# df["path"] = train_data_folder + df["filename"]

train_df = pd.read_csv('../input/laura-onac-dataset/fold_2_train.csv') 
train_df["path"] = train_data_folder + train_df["filename"] 
train_df['stalled'] = train_df['class']
train_df.drop(['class'], axis=1) 

val_df = pd.read_csv('../input/laura-onac-dataset/fold_2_test.csv') 
val_df["path"] = train_data_folder + val_df["filename"]
val_df['stalled'] = val_df['class']
val_df.drop(['class'], axis=1) 

# train_df, val_df = train_test_split(df, 
#                                     test_size=0.1, 
#                                     shuffle=True, 
#                                     stratify=df["stalled"].values, 
#                                     random_state=42)

print("Training data")
train_data_iterator = VideoIterator(train_df, transformations, device, True)
print("\nValidation data")
val_data_iterator = VideoIterator(val_df, transformations, device, False)

w = len(train_df[train_df["stalled"] == 0]) / len(train_df[train_df["stalled"] == 1])
print("\nStalled class weight: " + str(w))


#%% [code]

model_folder = "./models/model_v1/"
os.makedirs(model_folder, exist_ok=True)
# os.makedirs(model_folder)

no_epochs = 30
log_interval = 20

# Choose model architecture here:
# model = R3dModel().to(device)
# model = Rmc3Model().to(device)
model = R2plus1dModel().to(device)

# model_path = './best.pth'
# model.load_state_dict(torch.load(model_path, map_location=device))

criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters())
# this is actually cosine annealing
lr_scheduler = OneCycleLR(optimizer, pct_start=0.0001, max_lr=1e-4, epochs=no_epochs, steps_per_epoch=len(train_data_iterator))

#summary_writer = SummaryWriter(model_folder + "tensorlogs/")


#%% [code]
best_mcc = -1
best_loss = math.inf

if os.path.exists('./log.txt'):
    log_f = open('./log.txt', 'a')
else:
    log_f = open('./log.txt', 'w')

for i in range(no_epochs):
    
    model.train()
    partial_loss_sum = 0
    print("\nEpoch " + str(i))
    start_time = time.time()
    train_data_iterator.shuffle()
    train_losses = []
    for j, (x,x_back, y) in enumerate(train_data_iterator):
        outputs = model(x,x_back)
        optimizer.zero_grad()
        loss = criterion(outputs, y)
        train_losses.append(loss.item())
        partial_loss_sum += loss.item()
        if y[0].item() == 1:
            loss = loss * w
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        # Empty cache
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        if (j + 1) % log_interval == 0:
            end_time = time.time()
            total_time = round(end_time - start_time, 2)
            partial_loss = round(partial_loss_sum / log_interval, 6)
            lr = optimizer.param_groups[0]["lr"]
            print("    " + str(j + 1) + "/" + str(len(train_data_iterator)) + " | loss " + str(partial_loss) + " | lr " + str(lr) + " | " + str(total_time) + "s")
            start_time = time.time()
            partial_loss_sum = 0
    train_loss = round(sum(train_losses) / len(train_losses), 4)
                    
    print("\nEvaluating...")
    y_true = []
    y_pred = []
    y_pred_p = []

    with torch.no_grad():
        for j, (x,x_back,y) in enumerate(val_data_iterator):
            outputs = model(x,x_back)
            p = outputs[0].item()
            y_pred_p.append(p)
            if p < 0.5:
                p = 0
            else:
                p = 1
            y_pred.append(p)
            y_true.append(y[0].item())

    val_mcc = round(matthews_corrcoef(y_true, y_pred), 4)
    val_acc = round(accuracy_score(y_true, y_pred), 4)
    val_loss = round(log_loss(y_true, y_pred_p), 4)

    if val_mcc >= best_mcc or val_loss <= best_loss: 
        model_path = model_folder + "model_epoch:" + str(i) + "_mcc:" + str(val_mcc) + "_acc:" + str(val_acc) + "_loss" + str(val_loss) + ".pth"
        torch.save(model.state_dict(), model_path)
        if val_mcc >= best_mcc:
            best_mcc = val_mcc
        if val_loss <= best_loss:
            best_loss = val_loss

    print("Train loss: " + str(train_loss))
    print("Val loss: " + str(val_loss))
    print("Val MCC: " + str(val_mcc))
    print("Val accuracy: " + str(val_acc))  
    print("Train loss: " + str(train_loss),"Val loss: " + str(val_loss),"Val MCC: " + str(val_mcc),"Val accuracy: " + str(val_acc), "\n", file=log_f)  
log_f.close()
    # summary_writer.add_scalar("train_loss", train_loss, i)
    # summary_writer.add_scalar("val_loss", val_loss, i)
    # summary_writer.add_scalars("loss", {"train": train_loss, "val": val_loss}, i)
    # summary_writer.add_scalar("val_acc", val_acc, i)
    # summary_writer.add_scalar("val_mcc", val_mcc, i)
    # summary_writer.add_scalar("lr", optimizer.param_groups[0]["lr"], i)




# Inference

In [4]:
# import cv2
# import time
# import numpy as np
# import pandas as pd

# import torch
# import torchvision
# import torch.nn as nn
# from torchvision.transforms import transforms
# from torch.utils.data.dataset import Dataset

### Dataset

In [ ]:


# class VideoIterator(Dataset):
#     def __init__(self, data_csv, transforms, device):
#         self.df = data_csv
#         #self.df["path"] = data_dir + self.df["filename"]
#         self.transforms = transforms
#         self.device = device

#     def __getitem__(self, index):
#         row = self.df.iloc[index]
#         x = []
#         video = cv2.VideoCapture(row["path"])
#         if not video.isOpened():
#             print("Error opening video file!")
#         while video.isOpened():
#             ret, frame = video.read()
#             if ret:
#                 x.append(self.transforms(frame))
#             else:
#                 break
#         video.release()
#         x = torch.stack(x)
#         x = x.permute(1, 0, 2, 3)
#         x = x.unsqueeze(0)
#         x = x.to(self.device, dtype=torch.float)
#         y = torch.FloatTensor([[row["stalled"]]]).to(self.device)
#         return x, y

#     def __len__(self):
#         return len(self.df)



### Models

In [ ]:


# class R2plus1dModel(nn.Module):
#     def __init__(self):
#         super(R2plus1dModel, self).__init__()

#         self.cnn = torchvision.models.video.r2plus1d_18(pretrained=False)
#         self.cnn.fc = nn.Linear(in_features=512,
#                                 out_features=1)
#         self.sig = nn.Sigmoid()

#     def forward(self, input):
#         x = self.cnn(input)
#         x = self.sig(x)
#         return x
    

# class Rmc3Model(nn.Module):
#     def __init__(self):
#         super(Rmc3Model, self).__init__()

#         self.cnn = torchvision.models.video.mc3_18(pretrained=False)
#         self.cnn.fc = nn.Linear(in_features=512,
#                                 out_features=1)
#         self.sig = nn.Sigmoid()

#     def forward(self, input):
#         x = self.cnn(input)
#         x = self.sig(x)
#         return x


# class R3dModel(nn.Module):
#     def __init__(self):
#         super(R3dModel, self).__init__()

#         self.cnn = torchvision.models.video.r3d_18(pretrained=False)
#         self.cnn.fc = nn.Linear(in_features=512,
#                                 out_features=1)
#         self.sig = nn.Sigmoid()

#     def forward(self, input):
#         x = self.cnn(input)
#         x = self.sig(x)
#         return x



## Testing and validating

In [8]:


# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(device)

# transformations = transforms.Compose([
#                   transforms.ToTensor(),
#                   transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989])
# ])

# data_folder = "../input/alzheimer-nano-orange-blue/nano_orangeblue_resized_all"
# test_data_folder = data_folder + "/"

# test_df = pd.read_csv('../input/weight-files-valid-test-set/fold_2_validation.csv') 
# test_df["path"] = test_data_folder + test_df["filename"] 
# test_df['stalled'] = test_df['class']


# data_iterator = VideoIterator(test_df, transformations, device)
# print(len(data_iterator))

# model = R2plus1dModel().to(device)
# #model = Rmc3Model().to(device)
# #model = R3dModel().to(device)

# model_path = './best.pth'

# model.load_state_dict(torch.load(model_path, map_location=device))
# model.train()




# y_pred = []
# y_true = []

# with torch.no_grad():
#     for j, (x,x_back, y) in enumerate(data_iterator):
#         if j % 20 == 0:
#             print(j)
#         outputs = model(x, x_back)
#         p = outputs[0].item()
#         y_pred.append(p)
#         y_true.append(y[0].item())
        
# y_pred  = [0 if i<0.5 else 1 for i in y_pred]
# print("Done")

In [9]:
# from sklearn.metrics import matthews_corrcoef, accuracy_score, log_loss

# val_mcc = round(matthews_corrcoef(y_true, y_pred), 4)
# val_acc = round(accuracy_score(y_true, y_pred), 4)


# print("Test/Validation MCC =  ", val_mcc)
# print("Test/Validation Acc =  ", val_acc)
